<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Alpha_Beta_Crown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# α,β-CROWN ML-Verification Tutorial

*This tutorial shows the robustness verification of a neural network trained on the MNIST dataset with use of α,β-CROWN.*

**α,β-CROWN**
α,β-CROWN is an open-source neural network verifier. The code can be found on [their website](https://github.com/huanzhang12/alpha-beta-CROWN). 

In this tutorial we will first go through the installation of α,β-CROWN and train our own network. This network is to be verified later in the tutorial. Therefore, the verfication must be configured first. 

## Installation

This installation is based on another [tutorial](https://colab.research.google.com/drive/1mJTOmq2qHxMycHUzBepBN47QWcxda3ov#scrollTo=Y0toepwVIFTG). 

First we will set up our miniconda environment. 

In [ ]:
%%capture
%%bash
%env PYTHONPATH=
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Python in version 3.7 is installed into the environment. 

In [ ]:
%%capture
%%bash
conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

In [ ]:
%%capture
import sys
sys.path
!ls /usr/local/lib/python3.7/dist-packages
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

In order to use the library, we have to clone the corresponding git-repository.

In [ ]:
%%capture
!git clone https://github.com/huanzhang12/alpha-beta-CROWN.git

The environment is created. 

In [ ]:
%%capture
%%bash
# Remove the old environment, if necessary.
conda env remove --name alpha-beta-crown
conda env create -f alpha-beta-CROWN/complete_verifier/environment.yml  # install all dependents into the alpha-beta-crown environment

In [ ]:
%cd alpha-beta-CROWN/complete_verifier/

As explained on [the website of the α,β-project](https://github.com/huanzhang12/alpha-beta-CROWN), it is nessasary to create a configuration file in order to load the data. 

## Training of own network

In this section we create our own ML-model, calles my_model. This model is written into the file my_model.py. 
The network consists of three linear layers with ReLU activation function. 
Later on, we can use this model in our verification.  

In [ ]:
%%capture

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/mnist_trainer.py'}

In [ ]:
%%writefile my_model.py
import mnist_trainer
import torch
from torch import nn

def my_model(input_dim, output_dim, number_of_neurons):
    """Simple network with three linear layers and a ReLU activation function."""
    model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )
    return model

## Configure verification

Now we have created our model, but this model needs to be verified in the next step. Therefore, we create a file called my_config.yaml which configures parameters for verification. 

In [ ]:
%%writefile exp_configs/my_config.yaml
# This is an example configuration file that contains most useful parameter settings.
general:
  mode: verified-acc  # Compute verified accuracy.
model:
  # Use my_model() model in "my_model.py".
  name: Customized("my_model", "my_model", input_dim=10, output_dim=12, number_of_neurons=20)
  path: models/cifar10_resnet/resnet2b.pth  # Path to PyTorch checkpoint.
data:
  dataset: MNIST_ERAN_UN # Dataset name.
  std: [1.0] # Std for normalization.
  mean: [0.0] # Mean for normalization.
specification:
  norm: .inf  # Linf norm (can also be 2 or 1).
  epsilon: 0.00784313725490196  # epsilon=2./255.
attack:  # Currently attack is only implemented for Linf norm.
  pgd_steps: 100  # Increase for a stronger attack. A PGD attack will be used before verification to filter on non-robust data examples.
  pgd_restarts: 30  # Increase for a stronger attack.
solver:
  alpha-crown:
    iteration: 100   # Number of iterations for alpha-CROWN optimization. Alpha-CROWN is used to compute all intermediate layer bounds before branch and bound starts.
    lr_alpha: 0.1    # Learning rate for alpha in alpha-CROWN. The default (0.1) is typically ok.
  beta-crown:
    batch_size: 2048  # Number of subdomains to compute in parallel in beta-CROWN. Increase if you run out of memory.
    lr_alpha: 0.01  # Learning rate for optimizing the alpha parameters, the default (0.01) is typically ok, but you can try to tune this parameter to get better lower bound. 
    lr_beta: 0.05  # Learning rate for optimizing the beta parameters, the default (0.05) is typically ok, but you can try to tune this parameter to get better lower bound.
    iteration: 20  # Number of iterations for beta-CROWN optimization. 20 is often sufficient, 50 or 100 can also be used.
bab:
  timeout: 120  # Timeout threshold for branch and bound. Increase for verifying more points.
  branching:  # Parameters for branching heuristics.
    reduceop: min  # Reduction function for the branching heuristic scores, min or max. Using max can be better on some models.
    method: kfsb  # babsr is fast but less accurate; fsb is slow but most accurate; kfsb is usualy a balance.
    candidates: 3  # Number of candidates to consider in fsb and kfsb. More leads to slower but better branching. 3 is typically good enough.

## Verification of own network with α,β-CROWN

Now we only have to run our verification. 

First, we activate our environment.

In [ ]:
%source activate alpha-beta-crown

Then we call the robustness_verifier on our configured yaml-file. 

In [ ]:
%python robustness_verifier.py --config exp_configs/my_config.yaml --start 3 --end 4

We finish by deactivating the environment. 

In [ ]:
%conda deactivate